# **Recency (R), Frequency (F) and Monetary (M) Segmentation of Customers using Online Retail Dataset**

# Business problem

### An e-commerce company wants to divide its customers into groups and show marketing approaches specific to these groups. For this, it wants to divide the customers into groups by using various techniques and to exhibit various approaches specific to these groups.

# Dataset story
### The dataset named Online Retail includes online sales transactions of a UK-based retail company between 01/12/2009 and 09/12/2011. The company's product catalog includes souvenirs and it is known that most of its customers are wholesalers.¶
* InvoiceNo: Invoice Number (If this code starts with C, it means that the transaction has been cancelled)
* StockCode: Product code (unique for each product)
* Description: Product name
* Quantity: Number of products (How many of the products on the invoices were sold)
* InvoiceDate: Invoice date
* UnitPrice: Invoice price ( Sterling )
* CustomerID: Unique customer number
* Country: Country name

# Importing the libraries

In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Importing the dataset

In [2]:
df_ = pd.read_excel('/kaggle/input/online-retail-dataset/online_retail_II.xlsx', sheet_name='Year 2010-2011')
df = df_.copy()
df.columns = [col.lower() for col in df.columns]
df

,invoice,stockcode,description,quantity,invoicedate,price,customer id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.100,12680.000,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.150,12680.000,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.150,12680.000,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.950,12680.000,France


# Understanding the data

## Let's check dataframe

In [3]:
def check_dataframe(dataframe, head=5):
    print('\n', '#' * 20, 'head'.upper(), 20 * '#')
    print(dataframe.head(head))
    print('\n', '#' * 20, 'tail'.upper(), 20 * '#')
    print(dataframe.tail(head))
    print('\n', '#' * 20, 'shape'.upper(), 20 * '#')
    print(dataframe.shape)
    print('\n', '#' * 20, 'dtypes'.upper(), 20 * '#')
    print(dataframe.dtypes)
    print('\n', '#' * 20, 'columns'.upper(), 20 * '#')
    print(dataframe.columns)
    print('\n', '#' * 20, 'info'.upper(), 20 * '#')
    print(dataframe.info())
    print('\n', '#' * 20, 'any null values'.upper(), 20 * '#')
    print(dataframe.isnull().values.any())
    print('\n', '#' * 20, 'null values'.upper(), 20 * '#')
    print(dataframe.isnull().sum().sort_values(ascending=False))
    print('\n', '#' * 20, 'descriptive statistics'.upper(), 20 * '#')
    print(dataframe.describe([0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T)


check_dataframe(df)


 #################### HEAD ####################
  invoice stockcode                          description  quantity         invoicedate  price  customer id         country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00  2.550    17850.000  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00  3.390    17850.000  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00  2.750    17850.000  United Kingdom
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00  3.390    17850.000  United Kingdom
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6 2010-12-01 08:26:00  3.390    17850.000  United Kingdom

 #################### TAIL ####################
       invoice stockcode                      description  quantity         invoicedate  price  customer id country
541905  581587     22899     CHILDREN'S APRON DO

## Let's grab the categorical, numerical, and cardinal variables

In [4]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    # categorical variables
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == 'O']
    num_but_cat = [col for col in dataframe.columns if
                   dataframe[col].nunique() < cat_th and dataframe[col].dtypes != 'O']
    cat_but_car = [col for col in dataframe.columns if
                   dataframe[col].nunique() > car_th and dataframe[col].dtypes == 'O']
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # numerical variables
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != 'O']
    num_cols = [col for col in num_cols if col not in num_but_cat]

    # reporting section
    print(f'Observations: {dataframe.shape[0]}')
    print(f'Variables: {dataframe.shape[1]}')
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    # keeping the calculated values
    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 541910
Variables: 8
cat_cols: 0
num_cols: 4
cat_but_car: 4
num_but_cat: 0


In [5]:
cat_cols

[]

In [6]:
num_cols
num_cols = [col for col in num_cols if col not in ['invoicedate', 'customer id']]
num_cols

['quantity', 'price']

In [7]:
cat_but_car

['invoice', 'stockcode', 'description', 'country']

## Let's examine the cardinal variables

In [8]:
# Let's examine the frequence of the variable country
df['country'].value_counts()

United Kingdom          495478
Germany                   9495
France                    8558
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [9]:
# Let's examine the frequence of the variable stockcode
df['stockcode'].value_counts()

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
20738        1
62095B       1
72814        1
84967B       1
84546        1
Name: stockcode, Length: 4070, dtype: int64

In [10]:
# Let's examine the frequence of the variable description
df['description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER     2369
REGENCY CAKESTAND 3 TIER               2200
JUMBO BAG RED RETROSPOT                2159
PARTY BUNTING                          1727
LUNCH BAG RED RETROSPOT                1638
                                       ... 
Missing                                   1
historic computer difference?....se       1
DUSTY PINK CHRISTMAS TREE 30CM            1
WRAP BLUE RUSSIAN FOLKART                 1
PINK BERTIE MOBILE PHONE CHARM            1
Name: description, Length: 4223, dtype: int64

In [11]:
# Let's get the total number of products sold, the total price of the products sold, the stock codes of the invoices and the number of countries in the description breakdown.
df.groupby('description').agg(
    {'quantity': 'sum',
    'price': 'sum',
    'invoicedate': 'count',
    'stockcode': 'count',
    'country': 'count'
    }).sort_values('quantity', ascending=False)

,quantity,price,invoicedate,stockcode,country
description,,,,,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847,173.390,542,542,542
JUMBO BAG RED RETROSPOT,47363,5333.300,2159,2159,2159
ASSORTED COLOUR BIRD ORNAMENT,36381,2584.770,1501,1501,1501
POPCORN HOLDER,36334,874.500,865,865,865
PACK OF 72 RETROSPOT CAKE CASES,36039,1047.040,1385,1385,1385
...,...,...,...,...,...
Damaged,-7540,0.000,14,14,14
Printing smudges/thrown away,-9058,0.000,1,1,1
check,-12030,0.000,159,159,159


## Let's calculate the total price

In [12]:
df['total_price'] = df['price'] * df['quantity']
df

,invoice,stockcode,description,quantity,invoicedate,price,customer id,country,total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.100,12680.000,France,12.600
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.150,12680.000,France,16.600
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.150,12680.000,France,16.600
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.950,12680.000,France,14.850


## Let's examine the invoice varibale with total price

In [13]:
df.groupby('invoice').agg({'total_price': 'sum'}).sort_values('total_price', ascending=False)

,total_price
invoice,
581483,168469.600
541431,77183.600
574941,52940.940
576365,50653.910
556444,38970.000
...,...
C580605,-17836.460
C550456,-22998.400
C556445,-38970.000


# Data preparation

In [14]:
# Let's observ the missing data
df.isnull().sum().sort_values(ascending=False)

customer id    135080
description      1454
invoice             0
stockcode           0
quantity            0
invoicedate         0
price               0
country             0
total_price         0
dtype: int64

In [15]:
# Let's remove the missing data
df.dropna(inplace=True)

In [16]:
# Let's delete the observations with the invoices with 'C'
df = df[~df['invoice'].str.contains('C', na=False)]
df.shape

(397925, 9)

In [17]:
# The price can not zero. Let's fix this
df = df[df['price'] > 0]
df.describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,397885.000,12.988,179.332,1.000,2.000,6.000,12.000,80995.000
price,397885.000,3.117,22.098,0.001,1.250,1.950,3.750,8142.750
customer id,397885.000,15294.417,1713.144,12346.000,13969.000,15159.000,16795.000,18287.000
total_price,397885.000,22.397,309.071,0.001,4.680,11.800,19.800,168469.600


# Calculation of RFM metrics (recency, frequency, monetary)

## Let's determine the last date in the variable invoicedate

In [18]:
df['invoicedate'].max()

Timestamp('2011-12-09 12:50:00')

## Let's add 2 days to the last day of the variable invoicedate

In [19]:
analysis_date = dt.datetime(2011, 12, 11)

## Let's create a new datafrema named 'rfm' to be able to calculate the rfm metrics

In [20]:
rfm = df.groupby('customer id').agg({
    'invoicedate': lambda x: (analysis_date - x.max()).days,
    'invoice': lambda x: x.nunique(),
    'total_price': lambda x: x.sum()
})
rfm.head()

,invoicedate,invoice,total_price
customer id,,,
12346.000,326,1,77183.600
12347.000,3,7,4310.000
12348.000,76,4,1797.240
12349.000,19,1,1757.550
12350.000,311,1,334.400


## Let's change the column names with recency, frequency and monetary

In [21]:
rfm.columns = ['recency', 'frequency', 'monetary']
rfm.head()

,recency,frequency,monetary
customer id,,,
12346.000,326,1,77183.600
12347.000,3,7,4310.000
12348.000,76,4,1797.240
12349.000,19,1,1757.550
12350.000,311,1,334.400


## Monetary variable must not be lower than and equal to 0. Let's fix this


In [22]:
rfm = rfm[rfm['monetary'] > 0]
rfm

,recency,frequency,monetary
customer id,,,
12346.000,326,1,77183.600
12347.000,3,7,4310.000
12348.000,76,4,1797.240
12349.000,19,1,1757.550
12350.000,311,1,334.400
...,...,...,...
18280.000,278,1,180.600
18281.000,181,1,80.820
18282.000,8,2,178.050


## Let's calculate the rfm scores including recency, frequency, and monetary scores

In [23]:
rfm['recency_score'] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['monetary_score'] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm

,recency,frequency,monetary,recency_score,monetary_score,frequency_score
customer id,,,,,,
12346.000,326,1,77183.600,1,5,1
12347.000,3,7,4310.000,5,5,5
12348.000,76,4,1797.240,2,4,4
12349.000,19,1,1757.550,4,4,1
12350.000,311,1,334.400,1,2,1
...,...,...,...,...,...,...
18280.000,278,1,180.600,1,1,2
18281.000,181,1,80.820,1,1,2
18282.000,8,2,178.050,5,1,3


## Let's determine rf and rfm scores

In [24]:
rfm['rf_score'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)
rfm['rfm_score'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str)
rfm

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,rf_score,rfm_score
customer id,,,,,,,,
12346.000,326,1,77183.600,1,5,1,11,115
12347.000,3,7,4310.000,5,5,5,55,555
12348.000,76,4,1797.240,2,4,4,24,244
12349.000,19,1,1757.550,4,4,1,41,414
12350.000,311,1,334.400,1,2,1,11,112
...,...,...,...,...,...,...,...,...
18280.000,278,1,180.600,1,1,2,12,121
18281.000,181,1,80.820,1,1,2,12,121
18282.000,8,2,178.050,5,1,3,53,531


## Let's create rfm segments

In [25]:
segment_map = {r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose_them',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'}

rfm['segments'] = rfm['rf_score'].replace(segment_map, regex=True)
rfm

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,rf_score,rfm_score,segments
customer id,,,,,,,,,
12346.000,326,1,77183.600,1,5,1,11,115,hibernating
12347.000,3,7,4310.000,5,5,5,55,555,champions
12348.000,76,4,1797.240,2,4,4,24,244,at_risk
12349.000,19,1,1757.550,4,4,1,41,414,promising
12350.000,311,1,334.400,1,2,1,11,112,hibernating
...,...,...,...,...,...,...,...,...,...
18280.000,278,1,180.600,1,1,2,12,121,hibernating
18281.000,181,1,80.820,1,1,2,12,121,hibernating
18282.000,8,2,178.050,5,1,3,53,531,potential_loyalists


## Let's examine the mean and count values of the scores according to segments

In [26]:
rfm.groupby('segments').agg(
    {'recency': ['mean', 'count'],
    'frequency': ['mean', 'count'],
    'monetary': ['mean', 'count']})

recency       frequency       monetary      
                       mean count      mean count     mean count
segments                                                        
about_to_sleep       53.312   352     1.162   352  471.994   352
at_risk             153.786   593     2.877   593 1084.535   593
cant_loose_them     132.968    63     8.381    63 2796.156    63
champions             6.362   633    12.414   633 6857.964   633
hibernating         217.605  1071     1.102  1071  488.643  1071
loyal_customers      33.608   819     6.480   819 2864.248   819
need_attention       52.428   187     2.326   187  897.628   187
new_customers         7.429    42     1.000    42  388.213    42
potential_loyalists  17.399   484     2.010   484 1041.222   484
promising            23.511    94     1.000    94  294.008    94

### The values obtained with this above coding are actual values rather than scores. The administrator can request the 'need_attention' class. Because if this class does not receive attention, it may leave it to the e-commerce company that it is shopping with, or it may want to deal specifically with the 'at_risk' and 'can't loose' classes. In this case, what needs to be done is to select them and send the index information to the relevant department or write it to a database. Thus, these people can be accessed and related communications can be provided.

# Let's reach to the indexes of new_customers, cant_loose_them, and need_attention

In [27]:
rfm[rfm['segments'] == 'new_customers'].index
rfm[rfm['segments'] == 'cant_loose_them'].index
rfm[rfm['segments'] == 'need_attention'].index

new_df1 = pd.DataFrame()
new_df1['new_customer_id'] = rfm[rfm['segments'] == 'new_customers'].index
new_df1['new_customer_id'] = new_df1['new_customer_id'].astype(int)
new_df2 = pd.DataFrame()
new_df2['cant_loose_them_id'] = rfm[rfm['segments'] == 'cant_loose_them'].index
new_df2['cant_loose_them_id'] = new_df2['cant_loose_them_id'].astype(int)
new_df3 = pd.DataFrame()
new_df3['need_attention_id'] = rfm[rfm['segments'] == 'need_attention'].index
new_df3['need_attention_id'] = new_df3['need_attention_id'].astype(int)

In [28]:
print(new_df1.head(), '\n\n', new_df2.head(), '\n\n', new_df3.head())

   new_customer_id
0            12367
1            12442
2            12478
3            12479
4            12558 

    cant_loose_them_id
0               12643
1               12853
2               12868
3               12947
4               12980 

    need_attention_id
0              12360
1              12372
2              12413
3              12456
4              12536


## Let's get the final format of the new_df1, new_df2, new_df3, and rfm as csv file
#### new_df1.to_csv('new_customers.csv')
#### new_df2.to_csv('cant_loose_them.csv')
#### new_df3.to_csv('need_attention.csv')
#### rfm.to_csv('rfm.csv')

### Consequently, this analysis must be repeated from time to time. For example, when we create these segments in a certain month of a certain year, the customers in these segments may change in the next month, 3 months or 6 months. Therefore, it is very practical and valuable to observe the changes here. For example, this process should be able to run regularly every month. It should be able to report the changes in the segments that occur after running it every month and take action regarding the report after sending it to the relevant department. However, these processes should be followed continuously.

# **Thanks for checking my notebook!**